In [1]:
import matplotlib.pyplot as plt
import math
import numpy as np
import cv2
import scipy.optimize as coolNonLinOptimizer

# Exercise 5

These exercises will take you through: \ 
1. non-linear optimization, where you will implement a non-linear triangulation
2. checkerboard calibration, in real life with OpenCV.


## Nonlinear optimization
This exercise will take you through doing nonlinear optimization for triangulation of a single \
point. The same principles can be applied to more complex situations such as camera calibration, \
or situations where we lack a linear algoritm.


In [2]:
### Construct two camera matrices
R1 = np.identity(3)
R2 = np.identity(3)
t1 = np.array([[0, 0, 1]]).T
t2 = np.array([[0, 0, 20]]).T

K1 = np.array([[700, 0, 600], [0, 700, 400], [0, 0, 1]])
K2 = np.array([[700, 0, 600], [0, 700, 400], [0, 0, 1]])

In [3]:
### Both cameras observe the 3D point Q
Q = np.array([[1, 1, 0]]).T
Qh = np.vstack((Q, np.ones(len(Q[0]))))

### 5.1 Projection matrices

In [4]:
### 
P1 = K1@np.hstack((R1, t1))
P2 = K2@np.hstack((R2, t2))

### Projections
q1h = P1 @ Qh
q2h = P2 @ Qh
### Inhomogenous coordinates
q1 = q1h[0:2, :]/q1h[2, :]
q2 = q2h[0:2, :]/q2h[2, :]

In [5]:
print(q1,"\n", q2)

[[1300.]
 [1100.]] 
 [[635.]
 [435.]]


### 5.2 Simulated noise

In [6]:
q1_noisy = q1 + np.array([[1, -1]]).T
q2_noisy = q2 + np.array([[1, -1]]).T

Imported helper functions

In [7]:
def projectpoints(K, R, t, Q):
    """
    Projects 3D points of an object to the 2D image plane of the camera.
    Using homogenous coordinates the process can be done like this:
            p_h = K*[R t]*P_h

    Parameters:
        - K: camera matrix - hold intrinsic camera info like focault distance and principal points
        - R, t: pose of camera transformation; scale and transport object to the camera plane.
        - Q: 3xn the n 3D points to be projected onto image plane.
    
    Returns: 2xn matrix of projected points
    """
    
    # First creates the [R t] Matrix
    A = np.hstack((R, t))
    # Then, translate Q to homogenous plane => 4xn matrix by adding s=1
    B = np.vstack((Q, np.ones(len(Q[0]))))
    # Solve the projection in homogenous plane 
    p_h = K@A@B
    # Translate back to cartesian coordinates and return (divide all by s, then remove s)
    return p_h[0:2, :]/p_h[2, :]

def triangulate(q, P):
    """
    Return the traingulation.
    
    Parameters
    ----------
    q: 2 x n numpy array
        INHomogenous pixel coordinates q1... qn
        One for each camera seeing the point.
        At least two.
    P: list of 3 x 4 numpy arrays
        Projection matrices P1... Pn
        For each pixel coordinate
    
    Return
    ------
    Q: 3 x 1 numpy array
        Triangulation of the point using the linear SVD algorithm
    """
    _, n = q.shape # n = no. cameras has seen pixel.

    # Prepare B matrix. Two rows for each camera n.
    B = np.zeros((2 * n, 4))
    for i in range(n):
        B[2 * i: 2 * i + 2] = [
            P[i][2, :] * q[0, i] - P[i][0, :],
            P[i][2, :] * q[1, i] - P[i][1, :],
        ]
    # BQ = 0. Minimize using Svd.
    _, _, vh = np.linalg.svd(B)
    Q = vh[-1, :] # Q is ev. corresponding to the min. singular point.
    return Q[:3].reshape(3, 1) / Q[3] # Reshape and scale.

In [8]:
Q_projected = triangulate(np.hstack((q1_noisy, q2_noisy)), [P1, P2])

In [9]:
print(Q_projected)
print("Error/distance between Q and Q_tilde:", np.linalg.norm(Q-Q_projected))

[[1.01527507e+00]
 [9.85270570e-01]
 [2.85786810e-04]]
Error/distance between Q and Q_tilde: 0.021221817353380575


In [10]:
### Then reproject the points...
Qh_projected  = np.vstack((Q_projected, np.ones(len(Q_projected[0]))))
q1h_reprojected = P1 @ Qh_projected
q2h_reprojected = P2 @ Qh_projected
### Inhomogenous coordinates
q1_reprojected = q1h_reprojected[0:2, :]/q1h_reprojected[2, :]
q2_reprojected = q2h_reprojected[0:2, :]/q2h_reprojected[2, :]

In [11]:
print("Total distance of the noisy points and their reprojected points")
print("Camera1:", np.linalg.norm(q1_reprojected-q1_noisy)," pixels\nCamera2:", np.linalg.norm(q2_reprojected-q2_noisy), "pixels")

Total distance of the noisy points and their reprojected points
Camera1: 13.433018988191378  pixels
Camera2: 0.6717725840473774 pixels


### 5.3 Nonlinear triangulation

We are going to make a new function triangulate_nonlin that does triangulation using nonlinear \
optimization. It should take the same inputs as triangulate, i.e. a list of n pixel coordinates (q1, \
q2, . . . , qn), and a list of n projection matrices (P1, P2, . . . , Pn). 

Start by defining a helper-function inside triangulate_nonlin. \
This function, called compute_residuals, should take the parameters we want to optimize (in \
this case Q) as input, and should returns a vector of residuals (i.e. the numbers that we want to \
minimize the sum of squares of).

In [12]:
def triangulate_nonlin(q, P):
    """
    Return the traingulation using nonlinear optimization.
    
    Parameters
    ----------
    q: 2 x n numpy array
        INHomogenous pixel coordinates q1... qn
        One for each camera seeing the point.
        At least two.
    P: list of 3 x 4 numpy arrays
        Projection matrices P1... Pn
        For each pixel coordinate
    
    Return
    ------
    Q: 3 x 1 numpy array
        Triangulation of the point using the linear SVD algorithm,
        combined with least square omptimizer
    """
    # Initial guess using SVD
    Q0 = triangulate(q, P)
    Q0 = Q0.reshape(3)
    
    def compute_residuals(Q):
        """
        In our case residuals is a vector of differences in the projections.
        """
        Qh = np.vstack((Q.reshape(3,1), 1))
        # n cameras
        n = len(q[0])
        residuals = np.zeros(shape=(2*n,))

        for i in range(n):
            qh_est = P[i] @ Qh
            q_est = qh_est[0:2, :]/qh_est[2, :]
            r = q_est - q[:,i].reshape(2, 1)

            residuals[2*i] = r[0]
            residuals[2*i+1] = r[1]


        return residuals
    Q = coolNonLinOptimizer.least_squares(compute_residuals, Q0)["x"].reshape(3,1)
    return  Q


### 5.4 Testing

In [13]:
Q_nonlin = triangulate_nonlin(np.hstack((q1_noisy, q2_noisy)), [P1, P2])

In [14]:
print("Using only linear method:",Q_projected.flatten())
print("Using nonlinear methods:", Q_nonlin.flatten())
print("Real Q:", Q.flatten())
print("Q_nonlin difference: ",  np.linalg.norm(Q_nonlin-Q))
print("Q_linear difference: ",  np.linalg.norm(Q_projected-Q))


Using only linear method: [1.01527507e+00 9.85270570e-01 2.85786810e-04]
Using nonlinear methods: [1.00153898e+00 9.98546324e-01 4.27512498e-05]
Real Q: [1 1 0]
Q_nonlin difference:  0.0021174154572877386
Q_linear difference:  0.021221817353380575


### Camera calibration using openCV

In the following exercises you will be calibrating your own camera. For this we suggest using a camera in your phone or similar. \
Disable HDR on your phone. If you have a phone with a wide angle camera, consider using this camera for the exercise (as more \
lens distortion is more fun and challenging). Remember to disable lens correction in your camera app before taking the pictures. \
If you get stuck with the OpenCV functions, start by looking it up in the OpenCV documentation.


### 5.5 Calibration target

Take one of the provided calibration targets or print your own. If you do not have access to a printer, \
showing the target on a laptop or tablet display is also an option, albeit less ideal due to the glass on top of the display, \
which can cause reflection and refraction. Using your calibration target, take pictures of it from many different angles. \
Make sure to have an image of it straight on and well lit, and try more extreme angles as well. Try to get every part \
of the frame covered. You should have around twenty images.


#### 5.6 Load and process images

In [15]:
def get_rgb(path):
    bgr_img = cv2.imread(path)
    b,g,r = cv2.split(bgr_img)       # get b,g,r
    image = cv2.merge([r,g,b])
    return image


In [16]:
images = []
for i in range(20):
    image = get_rgb("checkerboard/c%02d.jpg" %i)
    image = cv2.resize(image, (600, 400))
    images.append(image)

In [17]:

for img in images:
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    ret, corners = cv2.findChessboardCorners(gray, (7, 10))
    if ret == True:
        # Draw and display the corners
        img = cv2.drawChessboardCorners(img, (7, 10), corners,ret)
        cv2.imshow('img',img)
        cv2.waitKey(800)

cv2.destroyAllWindows()